In [1]:
from tensorflow.python.keras.layers import Input,Dense,Reshape,Lambda,RNN,Concatenate,StackedRNNCells
from tensorflow.python.keras.models import Model,load_model
from tensorflow.python.keras.utils import multi_gpu_model
from tensorflow.python.keras.optimizers import Adam,SGD
from tensorflow.python.keras.callbacks import LearningRateScheduler,Callback,ModelCheckpoint,TensorBoard

import tensorflow as tf
import datetime
import os
import numpy as np
import pickle

%env PYTHONHASHSEED=0
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)
os.environ["CUDA_VISIBLE_DEVICES"]="0"

/home/ai/anaconda3/envs/gpu/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ai/anaconda3/envs/gpu/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ai/anaconda3/envs/gpu/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ai/anaconda3/envs/gpu/lib/python3.7/site-packages/tensorflow/

env: PYTHONHASHSEED=0


/home/ai/anaconda3/envs/gpu/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ai/anaconda3/envs/gpu/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ai/anaconda3/envs/gpu/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ai/anaconda3/envs/gpu/lib/python3.7/site

In [6]:
from sklearn.model_selection import train_test_split
#features
n2v_start = np.load("start_emb_128.npy")
n2v_end = np.load("end_emb_128.npy")
lanes = np.load("lanes.npy")
oneway = np.load("oneway.npy")
roadtype = np.load("highway.npy")
maxspeed = np.load("maxspeed.npy")
nodetype = np.load("nodetype.npy")
#target
meanspeed = np.load("meanspeed_only.npy")

#split
n2v_start_train, n2v_start_test, n2v_end_train, n2v_end_test, lanes_train, lanes_test, oneway_train, oneway_test, roadtype_train, roadtype_test, maxspeed_train, maxspeed_test, nodetype_train, nodetype_test, meanspeed_train, meanspeed_test = train_test_split(n2v_start, n2v_end, lanes,oneway,roadtype,maxspeed,nodetype,meanspeed)
del n2v_start
del n2v_end
del lanes
del oneway
del roadtype
del maxspeed
del nodetype
del meanspeed

In [2]:
n2v_start_train=np.load('/home/fyp2/Ideaer/Madrid/split/n2v_start_train.npy')
n2v_start_test=np.load('/home/fyp2/Ideaer/Madrid/split/n2v_start_test.npy')

n2v_end_train=np.load('/home/fyp2/Ideaer/Madrid/split/n2v_end_train.npy')
n2v_end_test=np.load('/home/fyp2/Ideaer/Madrid/split/n2v_end_test.npy')

lanes_train=np.load('/home/fyp2/Ideaer/Madrid/split/lanes_train.npy')
lanes_test=np.load('/home/fyp2/Ideaer/Madrid/split/lanes_test.npy')

oneway_train=np.load('/home/fyp2/Ideaer/Madrid/split/oneway_train.npy')
oneway_test=np.load('/home/fyp2/Ideaer/Madrid/split/oneway_test.npy')

roadtype_train=np.load('/home/fyp2/Ideaer/Madrid/split/roadtype_train.npy')
roadtype_test=np.load('/home/fyp2/Ideaer/Madrid/split/roadtype_test.npy')

maxspeed_train=np.load('/home/fyp2/Ideaer/Madrid/split/maxspeed_train.npy')
maxspeed_test=np.load('/home/fyp2/Ideaer/Madrid/split/maxspeed_test.npy')

nodetype_train=np.load('/home/fyp2/Ideaer/Madrid/split/nodetype_train.npy')
nodetype_test=np.load('/home/fyp2/Ideaer/Madrid/split/nodetype_test.npy')

meanspeed_train=np.load('/home/fyp2/Ideaer/Madrid/split/meanspeed_train.npy')
meanspeed_test=np.load('/home/fyp2/Ideaer/Madrid/split/meanspeed_test.npy')

In [47]:
embinplist = list()
inplist = list()

n2v_start_inp = Input((128,))
n2v_start_reg = Dense(128,activation = "relu", use_bias = True)(n2v_start_inp)
inplist.append(n2v_start_inp)
embinplist.append(n2v_start_reg)

n2v_end_inp = Input((128,))
n2v_end_reg = Dense(128,activation = "relu", use_bias = True)(n2v_end_inp)
inplist.append(n2v_end_inp)
embinplist.append(n2v_end_reg)

lanes_inp = Input((1,))
lanes_reg= Dense(1,activation = "relu", use_bias = True)(lanes_inp)
inplist.append(lanes_inp)
embinplist.append(lanes_reg)
"""
oneway_inp = Input((2,))
oneway_reg= Dense(2,activation = "relu", use_bias = True)(oneway_inp)
inplist.append(oneway_inp)
embinplist.append(oneway_reg)
"""
roadtype_inp = Input((13,))
roadtype_reg= Dense(8,activation = "relu", use_bias = True)(roadtype_inp)
inplist.append(roadtype_inp)
embinplist.append(roadtype_reg)

maxspeed_inp = Input((1,))
maxspeed_reg= Dense(1,activation = "relu", use_bias = True)(maxspeed_inp)
inplist.append(maxspeed_inp)
embinplist.append(maxspeed_reg)
"""
nodetype_inp = Input((11,))
nodetype_reg= Dense(8,activation = "relu", use_bias = True)(nodetype_inp)
inplist.append(nodetype_inp)
embinplist.append(nodetype_reg)
"""
combine = Concatenate()(embinplist)
proj = Dense(32,activation = "relu",use_bias = True)(combine)
proj = Dense(1,activation = "linear")(proj)

emb = Model(inplist,proj)

In [4]:
#emb.summary()

In [48]:
adam=Adam(lr=0.01,beta_1=0.9,beta_2=0.999,amsgrad=True)
emb.compile(optimizer=adam, loss='mae',metrics=['mae'])

In [49]:
history =emb.fit([n2v_start_train,n2v_end_train,lanes_train,roadtype_train,maxspeed_train],
        meanspeed_train,
        epochs=100,batch_size=256,
        validation_data=([n2v_start_test,n2v_end_test,lanes_test,roadtype_test,maxspeed_test],
        meanspeed_test)
       )

Train on 86992 samples, validate on 28998 samples
Epoch 1/100
86992/86992 [==============================] - 3s 40us/sample - loss: 8.9378 - mean_absolute_error: 8.9378 - val_loss: 7.1441 - val_mean_absolute_error: 7.1441
Epoch 2/100
86992/86992 [==============================] - 3s 31us/sample - loss: 6.6343 - mean_absolute_error: 6.6343 - val_loss: 6.3422 - val_mean_absolute_error: 6.3422
Epoch 3/100
86992/86992 [==============================] - 3s 32us/sample - loss: 5.8202 - mean_absolute_error: 5.8202 - val_loss: 5.7523 - val_mean_absolute_error: 5.7523
Epoch 4/100
86992/86992 [==============================] - 3s 32us/sample - loss: 5.2799 - mean_absolute_error: 5.2799 - val_loss: 5.4699 - val_mean_absolute_error: 5.4699
Epoch 5/100
86992/86992 [==============================] - 3s 30us/sample - loss: 4.9153 - mean_absolute_error: 4.9153 - val_loss: 5.2071 - val_mean_absolute_error: 5.2071
Epoch 6/100
86992/86992 [==============================] - 3s 32us/sample - loss: 4.6271 -

86992/86992 [==============================] - 3s 31us/sample - loss: 2.7349 - mean_absolute_error: 2.7349 - val_loss: 4.0104 - val_mean_absolute_error: 4.0104
Epoch 49/100
86992/86992 [==============================] - 3s 31us/sample - loss: 2.7293 - mean_absolute_error: 2.7293 - val_loss: 4.0239 - val_mean_absolute_error: 4.0239
Epoch 50/100
86992/86992 [==============================] - 3s 31us/sample - loss: 2.7102 - mean_absolute_error: 2.7102 - val_loss: 4.0129 - val_mean_absolute_error: 4.0129
Epoch 51/100
86992/86992 [==============================] - 3s 30us/sample - loss: 2.7058 - mean_absolute_error: 2.7058 - val_loss: 3.9984 - val_mean_absolute_error: 3.9984
Epoch 52/100
86992/86992 [==============================] - 3s 31us/sample - loss: 2.7090 - mean_absolute_error: 2.7090 - val_loss: 4.1164 - val_mean_absolute_error: 4.1164
Epoch 53/100
86992/86992 [==============================] - 3s 32us/sample - loss: 2.7142 - mean_absolute_error: 2.7142 - val_loss: 3.9954 - val_mea

86992/86992 [==============================] - 3s 32us/sample - loss: 2.4295 - mean_absolute_error: 2.4295 - val_loss: 4.0257 - val_mean_absolute_error: 4.0257
Epoch 96/100
86992/86992 [==============================] - 3s 32us/sample - loss: 2.4158 - mean_absolute_error: 2.4158 - val_loss: 4.0826 - val_mean_absolute_error: 4.0826
Epoch 97/100
86992/86992 [==============================] - 3s 31us/sample - loss: 2.4092 - mean_absolute_error: 2.4092 - val_loss: 3.9715 - val_mean_absolute_error: 3.9715
Epoch 98/100
86992/86992 [==============================] - 3s 31us/sample - loss: 2.4071 - mean_absolute_error: 2.4071 - val_loss: 3.9679 - val_mean_absolute_error: 3.9679
Epoch 99/100
86992/86992 [==============================] - 3s 31us/sample - loss: 2.4112 - mean_absolute_error: 2.4112 - val_loss: 3.9574 - val_mean_absolute_error: 3.9574
Epoch 100/100
86992/86992 [==============================] - 3s 32us/sample - loss: 2.3967 - mean_absolute_error: 2.3967 - val_loss: 3.9550 - val_me

In [ ]:
#emb.save('/home/fyp2/Ideaer/Madrid/model/all')

In [ ]:
import sys
print(sys.path)

In [50]:
loss_hist = history.history['loss']
mae_history = history.history["val_mean_absolute_error"]
#print("min loss :"+str(np.min(loss_hist)))
#print("min val mae :"+str(np.min(mae_history)))
print(np.min(loss_hist))
print(np.min(mae_history))

2.3967475220629546
3.9449413
